In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [4]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
df = df.drop('Unnamed: 2', axis=1)
df = df.drop('Unnamed: 3', axis=1)
df = df.drop('Unnamed: 4', axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [7]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [8]:
X = df['v2']
y = df['v1']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Решить задачу классификации с понижением размерности. Использовать самостоятельно реализованные модели из предыдущих ЛР.

In [10]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, RFE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report

In [11]:
def test_class(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
  bag = BaggingClassifier().fit(X_train, y_train)
  print(classification_report(y_test, bag.predict(X_test)))

In [16]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Создание экземпляра CountVectorizer
vectorizer = CountVectorizer()

# Преобразование текстовых данных в матрицу признаков
X_bow = vectorizer.fit_transform(X)
from sklearn.feature_extraction.text import TfidfVectorizer

# Создание экземпляра TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Преобразование текстовых данных в матрицу признаков
X_tfidf = tfidf_vectorizer.fit_transform(X)
# 1. SelectKBest
selector = SelectKBest(k=2)  # количество наиболее значимых признаков
X_SelectKBest_classifier1 = selector.fit_transform(X_bow, y)
X_SelectKBest_classifier2 = selector.fit_transform(X_tfidf, y)

In [17]:
test_class(X_SelectKBest_classifier1, y)

              precision    recall  f1-score   support

         ham       0.95      0.96      0.96       917
        spam       0.66      0.60      0.63       117

    accuracy                           0.92      1034
   macro avg       0.80      0.78      0.79      1034
weighted avg       0.92      0.92      0.92      1034



In [18]:
test_class(X_SelectKBest_classifier2, y)

              precision    recall  f1-score   support

         ham       0.91      1.00      0.95       917
        spam       0.90      0.24      0.38       117

    accuracy                           0.91      1034
   macro avg       0.91      0.62      0.67      1034
weighted avg       0.91      0.91      0.89      1034



In [19]:
# 3. RFE (Recursive Feature Elimination)
estimator = BaggingClassifier()
selector = RFE(estimator, n_features_to_select=5)  # количество выбранных признаков (n_features_to_select)
X_RFE_classifier1 = selector.fit_transform(X_SelectKBest_classifier1, y)
X_RFE_classifier2 = selector.fit_transform(X_SelectKBest_classifier2, y)

In [20]:
test_class(X_RFE_classifier1, y)

              precision    recall  f1-score   support

         ham       0.95      0.96      0.96       917
        spam       0.66      0.60      0.63       117

    accuracy                           0.92      1034
   macro avg       0.80      0.78      0.79      1034
weighted avg       0.92      0.92      0.92      1034



In [21]:
test_class(X_RFE_classifier2, y)

              precision    recall  f1-score   support

         ham       0.91      1.00      0.95       917
        spam       0.90      0.24      0.38       117

    accuracy                           0.91      1034
   macro avg       0.91      0.62      0.67      1034
weighted avg       0.91      0.91      0.89      1034



In [22]:
# 6. Isomap
isomap = Isomap(n_components=10)  # количество компонент (n_components)
X_Isomap_classifier1 = isomap.fit_transform(X_bow)
X_Isomap_classifier2 = isomap.fit_transform(X_tfidf)

D:\environment\venv\lib\site-packages\sklearn\manifold\_isomap.py:373: UserWarning: The number of connected components of the neighbors graph is 6 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
D:\environment\venv\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
D:\environment\venv\lib\site-packages\sklearn\manifold\_isomap.py:373: UserWarning: The number of connected components of the neighbors graph is 4 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
D:\environment\venv\lib\site-packages\scipy\sparse\_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x

In [23]:
test_class(X_Isomap_classifier1, y)

              precision    recall  f1-score   support

         ham       0.94      0.98      0.96       917
        spam       0.75      0.55      0.63       117

    accuracy                           0.93      1034
   macro avg       0.85      0.76      0.80      1034
weighted avg       0.92      0.93      0.92      1034



In [24]:
test_class(X_Isomap_classifier2, y)

              precision    recall  f1-score   support

         ham       0.93      0.97      0.95       917
        spam       0.62      0.43      0.51       117

    accuracy                           0.91      1034
   macro avg       0.78      0.70      0.73      1034
weighted avg       0.90      0.91      0.90      1034



### Решить задачу тематического моделирования с помощью LDA.

In [26]:
pip install nltk

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
      --------------------------------------- 0.0/1.5 MB 1.3 MB/s eta 0:00:02
     -- ------------------------------------- 0.1/1.5 MB 1.1 MB/s eta 0:00:02
     ---- ----------------------------------- 0.2/1.5 MB 1.1 MB/s eta 0:00:02
     ----- ---------------------------------- 0.2/1.5 MB 1.1 MB/s eta 0:00:02
     --------- ------------------------------ 0.4/1.5 MB 1.5 MB/s eta 0:00:01
     -------------- ------------------------- 0.6/1.5 MB 1.9 MB/s eta 0:00:01
     ------------------- -------------------- 0.7/1.5 MB 2.2 MB/s eta 0:00:01
     -------------------------- ------------- 1.0/1.5 MB 2.6 MB/s eta 0:00:01
     --------------------------- ------------ 1.0/1.5 MB 2.7 MB/s eta 0:00:01
     --------------------------- ------------ 1.0/1.5 MB 2.2 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB 1.3 MB/s eta 0:00:19
   ---------------------------------------- 0.1/24.0 MB 1.1 MB/s eta 0:00:23
   ---------------------------------------- 0.2/24.0 MB 1.1 MB/s eta 0:00:22
   ---------------------------------------- 0.2/24.0 MB 1.1 MB/s eta 0:00:23
    --------------------------------------- 0.3/24.0 MB 1.4 MB/s eta 0:00:17
    --------------------------------------- 0.6/24.0 MB 2.1 MB/s eta 0:00:12
   - -------------------------------------- 0.8/24.0 MB 2.5 MB/s eta 0:00:10
   - -------------------------------------- 1.0/24.0 MB 2.7 MB/s eta 0:00:09
   - -------------------------------------- 1.0/24.0 MB 2.7 MB/s eta 0:00:09
   - -------------------------------------- 1.0/24.0 MB 2.7 MB/s eta 0:00:09
   - -------------------------------------- 1.0/24.0 MB 2.7 MB/s eta 0:00:09
   - -------------------------------------- 1.0/24.0 MB 2.7 MB/s eta 0:00:09
   - -


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Полина\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
# Создание объекта CountVectorizer для извлечения признаков
vectorizer = CountVectorizer(max_features=1000,   # Ограничение на максимальное количество признаков
                             stop_words='english',  # Игнорирование стоп-слов
                             max_df=0.5,  # Игнорирование терминов, которые появляются в более чем 50% документов
                             min_df=2)  # Игнорирование терминов, которые появляются в менее чем 2 документах

In [31]:
# Преобразование текстовых данных в матрицу признаков
X_features = vectorizer.fit_transform(X)

In [32]:
# Создание модели LDA
num_topics = 10  # Количество тем
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)

In [33]:
# Обучение модели LDA
lda_model.fit(X_features)

# Получение самых вероятных слов для каждой темы
feature_names = vectorizer.get_feature_names_out()
top_words = 10  # Количество верхних слов для вывода

In [34]:
for topic_idx, topic in enumerate(lda_model.components_):
    top_feature_indices = topic.argsort()[:-top_words - 1:-1]
    top_features = [feature_names[i] for i in top_feature_indices]
    print(f"Тема #{topic_idx + 1}:")
    print(top_features)
    print()

Тема #1:
['know', 'did', 'right', 'yes', 'just', 'tonight', 'say', 'gonna', 'phone', 'like']

Тема #2:
['lor', 'got', 'ok', 'oh', 'wat', 'pls', 'dun', 'wan', 'ask', 'ìï']

Тема #3:
['tell', 'just', 'don', 'yeah', 'sure', 'time', 'got', 'ok', 'going', 'll']

Тема #4:
['like', 'going', 'da', 'way', 'feel', 'think', 'time', 'ya', 'life', 'make']

Тема #5:
['claim', 'prize', 'won', 'number', 'cash', 'www', 'urgent', 'win', 'com', 'txt']

Тема #6:
['good', 'love', 'day', 'ì_', 'hi', 'hope', 'happy', 'morning', 'babe', 'miss']

Тема #7:
['ll', 'just', 'sorry', 'need', 'want', 'come', 'sent', 'dont', 'later', 'time']

Тема #8:
['txt', 'home', 'new', 'ur', 'chat', '150p', 'send', 'stop', 'week', 'free']

Тема #9:
['gt', 'lt', 'ur', 'ok', 'reply', 'msg', 'send', 'sms', 'heart', 'text']

Тема #10:
['free', 'text', 'know', 'mobile', 'stop', 'let', 'phone', 'reply', 'aight', 'txt']



In [35]:
# Получение распределения тем для каждого документа
topic_distribution = lda_model.transform(X_features)

In [36]:
most_likely_topics = topic_distribution.argmax(axis=1)

In [37]:
most_likely_topics

array([0, 3, 4, ..., 0, 0, 6], dtype=int64)

In [38]:
df['most_likely_topic'] = most_likely_topics

In [39]:
df

,v1,v2,most_likely_topic
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,3
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,4
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5164,spam,This is the 2nd time we have tried 2 contact u...,4
5165,ham,Will Ì_ b going to esplanade fr home?,5
5166,ham,"Pity, * was in mood for that. So...any other s...",0
5167,ham,The guy did some bitching but I acted like i'd...,0
